In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv
/kaggle/input/loan-approval-prediction/sample_submission.csv
/kaggle/input/loan-approval-prediction/train.csv
/kaggle/input/loan-approval-prediction/test.csv


In [26]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import uniform
from datetime import datetime
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from sklearn.preprocessing import LabelEncoder

In [27]:
train = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
test  = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s4e10/sample_submission.csv')

# 삭제 전 ID 저장
train_idx = train['id']
test_idx = test['id']

train = train.drop(columns=['id'])
test = test.drop(columns=['id'])

# 데이터에 파생변수1 추가
def cal_loan_mean_ratio(df):
    
    mean_lst = df.groupby('loan_grade')['loan_amnt'].mean()
    df['loan_amnt_mean_ratio'] = df.apply(lambda x: x['loan_amnt'] / mean_lst[x['loan_grade']], axis = 1)
    
    return df

train = cal_loan_mean_ratio(train)
test = cal_loan_mean_ratio(test)

'''
# 데이터에 파생변수 2 추가
# 가중치 검사 및 검증 결과 유의미하지 않음
def int_rate_by_Credlength(df):
    df['int_rate_by_Credlength'] = df['loan_int_rate'] / df['cb_person_cred_hist_length']
    return df

train = int_rate_by_Credlength(train)
test = int_rate_by_Credlength(test)
'''

# 데이터에 파생변수 3 추가
# 유효하지 않은 열을 유효하게 -> 나이/재직기간을 범주화 하자.

def categorize_age(age):
    if age < 20:
        return 0
    elif 20 <= age < 30:
        return 1
    elif 30 <= age < 40:
        return 2
    elif 40 <= age < 50:
        return 3
    else:
        return 4

train['age_to_categorial'] = train['person_age'].apply(categorize_age)
test['age_to_categorial'] = test['person_age'].apply(categorize_age)


# 데이터에 파생변수 4 추가
# 재직기간의 범주화
'''
def categorize_emp(emp):
    if emp < 3:
        return 0
    elif 3 <= emp < 6:
        return 1
    elif 6 <= emp < 10:
        return 2
    elif 10 <= emp < 20:
        return 3
    else:
        return 4

train['emp_to_categorial'] = train['person_emp_length'].apply(categorize_emp)
test['emp_to_categorial'] = test['person_emp_length'].apply(categorize_emp)
'''

"\ndef categorize_emp(emp):\n    if emp < 3:\n        return 0\n    elif 3 <= emp < 6:\n        return 1\n    elif 6 <= emp < 10:\n        return 2\n    elif 10 <= emp < 20:\n        return 3\n    else:\n        return 4\n\ntrain['emp_to_categorial'] = train['person_emp_length'].apply(categorize_emp)\ntest['emp_to_categorial'] = test['person_emp_length'].apply(categorize_emp)\n"

In [28]:
# 결측치는 러프하게 평균으로 진행

def Na_mean_change(df, features):
    
    for col in features:
        df[col].fillna(df[col].mean(), inplace = True)
    
    pd.DataFrame(df)
    return df

In [29]:
# 라벨 인코더 적용

def preprocess_data(df_train, df_test):
    label_enc = LabelEncoder()
    
    label_cols = ['person_home_ownership', 'loan_grade', 'cb_person_default_on_file']
    
    for col in label_cols:
        df_train[col] = label_enc.fit_transform(df_train[col])
        df_test[col] = label_enc.transform(df_test[col])
        
    df_train = pd.get_dummies(df_train, columns = ['loan_intent'], drop_first = True)
    df_test = pd.get_dummies(df_test, columns = ['loan_intent'], drop_first = True)
    
    target_col = 'loan_status'
    train_columns = df_train.drop(columns =[target_col]).columns
    
    df_test = df_test.reindex(columns = train_columns, fill_value = 0)
    
    return df_train, df_test

In [30]:
train2 = train
test2 = test

In [31]:
Na_mean_change(train2, ['person_emp_length', 'loan_int_rate'])
df_test = Na_mean_change(test2, ['person_emp_length', 'loan_int_rate'])

/tmp/ipykernel_30/800587267.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace = True)


In [32]:
X = train2.drop(columns = 'loan_status', axis = 1).iloc[:,:-5]
y = train2['loan_status']

# Futher work begins here

- use ML pipelines
- use miltiple ML methods
     - LightGBM, XGBoost, etc
 

In [33]:
# separate num cols and car cols

cat_cols = X.select_dtypes(exclude = np.number).columns.tolist()
num_cols = X.select_dtypes(include = np.number).columns.tolist()

num_cols

['person_age',
 'person_income',
 'person_emp_length',
 'loan_amnt',
 'loan_int_rate']

In [34]:
# 훈련 - 테스트 데이터 분리

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42
)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(46916, 8) (46916,) (11729, 8) (11729,)


In [35]:
# 훈련 - 검증 데이터 분리

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size = 0.2, random_state = 42
)

print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(37532, 8) (9384, 8) (37532,) (9384,)


In [36]:
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # 결측치 처리 관련
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier


# StratifiedKFold : 분류 작업 할 때
# KFold : 수치 작업 할 때
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold, KFold

import numpy as np
from scipy.stats import uniform, randint

In [ ]:
# 파이프라인 구축

# 연속형/범주형 피처 변환기 설정

categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer( # 최종 전처리
    transformers=[
        ('num', numerical_transformer, num_cols), # 이름, 처리 방법, 대상 순
        ('cat', categorical_transformer, cat_cols)
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), #전처리 라인 적용
    ('classifier', RandomForestClassifier(random_state=42)) #분류는 포레스트 적용
])

param_distributions = { #파라미터 설정
    'classifier__min_impurity_decrease': uniform(0.0001, 0.001),
    'classifier__max_depth': randint(20, 50),
    'classifier__min_samples_split': randint(2, 25),
    'classifier__min_samples_leaf': randint(1, 25),
}

split_number = 5  # fold숫자 정하기
stratified_kfold = StratifiedKFold(n_splits=split_number, shuffle=True, random_state=42)

random_search = RandomizedSearchCV( #랜덤서치 실행
    estimator=pipeline, #원래는 Tree, REG같은 모델이 들어갔던 자리, 전처리와 모델처리가 한번에 진행됨.
    param_distributions=param_distributions,
    n_iter=50, #시도횟수
    cv= stratified_kfold,
    scoring='roc_auc', # 평가 지표
    random_state=42,
    n_jobs=-1 #가용한 모든 코어 이용
)

random_search.fit(X_train, y_train)

# 검증 세트에 대한 예측 확률 계산
y_val_probs = random_search.predict_proba(X_val)[:, 1]  # 클래스 1(양성 클래스)에 대한 확률 추출

# ROC-AUC 점수 계산 및 출력
roc_auc = roc_auc_score(y_val, y_val_probs)
print(f'ROC-AUC 점수: {roc_auc:.4f}')

In [ ]:
#LightGBM 적용

from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# LightGBM이 적용된 파이프라인
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # 전처리 라인 적용
    ('classifier', LGBMClassifier(random_state=42))  # 분류는 LightGBM 적용
])

# LightGBM 파라미터 설정
param_distributions = {
    'classifier__learning_rate': uniform(0.01, 0.1),  # 학습률
    'classifier__n_estimators': randint(50, 500),  # 트리의 개수
    'classifier__max_depth': randint(3, 15),  # 트리의 최대 깊이
    'classifier__num_leaves': randint(20, 50),  # 리프 노드의 최대 개수
    'classifier__min_child_samples': randint(10, 50),  # 리프 노드의 최소 샘플 수
    'classifier__subsample': uniform(0.6, 0.4),  # 데이터 샘플링 비율
    'classifier__colsample_bytree': uniform(0.6, 0.4),  # 컬럼 샘플링 비율
    'classifier__reg_alpha': uniform(0.0, 0.1),  # L1 정규화
    'classifier__reg_lambda': uniform(0.0, 0.1),  # L2 정규화
}

# RandomizedSearchCV를 사용한 하이퍼파라미터 튜닝
random_search_2 = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=50,  # 시도 횟수
    cv=stratified_kfold,  # 교차 검증
    scoring='roc_auc',  # 평가 지표
    random_state=42,
    n_jobs=-1  # 가용한 모든 코어 사용
)

random_search_2.fit(X_train, y_train)

# 검증 세트에 대한 예측 확률 계산
y_val_probs = random_search_2.predict_proba(X_val)[:, 1]  # 클래스 1(양성 클래스)에 대한 확률 추출

# ROC-AUC 점수 계산 및 출력
roc_auc = roc_auc_score(y_val, y_val_probs)
print(f'ROC-AUC 점수: {roc_auc:.4f}')

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

def get_score(model, X_tr, X_val, y_tr, y_val):
    tr_pred = model.predict_proba(X_tr)[:, 1]
    val_pred = model.predict_proba(X_val)[:, 1]
    tr_score = np.sqrt(roc_auc_score(y_tr, tr_pred))
    val_score = np.sqrt(roc_auc_score(y_val, val_pred))
    return f"train: {tr_score}, validation: {val_score}"

best_model = random_search.best_estimator_
get_score(best_model, X_tr, X_val, y_tr, y_val)

In [ ]:
best_model = random_search_2.best_estimator_
get_score(best_model, X_tr, X_val, y_tr, y_val)

In [ ]:
final_preds = best_model.predict(X_test)
result = pd.DataFrame({'Approval': final_preds})
result.value_counts()

# Model Ensombling - 3models
- Tree, XGBoost, LightGBM 3 model ensemble

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score

# 데이터 준비

# 수치형 데이터 전처리
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 범주형 데이터 전처리
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 전처리기 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)

# 모델별 파이프라인 정의
models = {
    "LightGBM": LGBMClassifier(random_state=42),
    "RandomForestRegressor": RandomForestRegressor(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# 평가 결과 저장
results = {}

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 파이프라인 생성 및 모델 학습
for model_name, model in models.items():
    print(f"\n모델: {model_name}")
    
    # 파이프라인 생성
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    # 교차 검증을 통한 점수 계산
    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=stratified_kfold, scoring='roc_auc', n_jobs=-1)
    print(f"교차 검증 평균 ROC-AUC 점수: {np.mean(cv_scores):.4f}")
    
    # 학습 및 검증
    pipeline.fit(X_train, y_train)
    y_val_probs = pipeline.predict_proba(X_val)[:, 1]  # 클래스 1에 대한 확률 추출
    roc_auc = roc_auc_score(y_val, y_val_probs)
    print(f"검증 세트 ROC-AUC 점수: {roc_auc:.4f}")
    
    # 결과 저장
    results[model_name] = {
        'CV ROC-AUC': np.mean(cv_scores),
        'Validation ROC-AUC': roc_auc
    }

# 최종 결과 출력
print("\n모델별 평가 결과:")
for model_name, scores in results.items():
    print(f"{model_name}:")
    print(f"  - 교차 검증 평균 ROC-AUC: {scores['CV ROC-AUC']:.4f}")
    print(f"  - 검증 세트 ROC-AUC: {scores['Validation ROC-AUC']:.4f}")

In [ ]:
def get_score(model, X_tr, X_val, y_tr, y_val):
    """
    주어진 모델에 대해 훈련 및 검증 데이터에서 ROC-AUC 점수를 계산합니다.
    """
    # 예측 확률 계산
    tr_pred = model.predict_proba(X_tr)[:, 1]
    val_pred = model.predict_proba(X_val)[:, 1]
    
    # 점수 계산
    tr_score = np.sqrt(roc_auc_score(y_tr, tr_pred))
    val_score = np.sqrt(roc_auc_score(y_val, val_pred))
    
    return f"Train Score: {tr_score:.4f}, Validation Score: {val_score:.4f}"

# 최적 모델에 대해 점수 출력
for model_name, model_pipeline in models.items():
    print(f"\n모델: {model_name}")
    
    # 전처리를 포함한 파이프라인으로 학습
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model_pipeline)
    ])
    pipeline.fit(X_train, y_train)  # 데이터 학습
    
    # 성능 평가
    try:
        print(get_score(pipeline, X_train, X_val, y_train, y_val))
    except AttributeError as e:
        print(f"모델 {model_name}은 'predict_proba'를 지원하지 않습니다.")